# 07 Systems Security Shellcode

Compile assembly with GCC: 

```gcc -m64 -nostdlib -no-pie -o hello64.bin hello64.s```

Extract only the machine code from generated code:

```objcopy -o binary --only-section=.text hello64.bin hello64.s```

View bytes in hex (encode them in a string):

```hexdump -v hello64.sc -e '"\\""x"1/1"%02x"""'```

For anything useful system calls are required:

- Linux: system call API is powerful and easy
  - 32-bit: ```0x80 int``` instruction is used to call system calls
    - system call number is passed to register %eax
    - %eax contains result from the system call
  - 64-bit: ```syscall``` instruction is used
    - system call number is passed to %rax
- Windows: API is hard, so library functions are preferred
- Calling functions is easy if you know the offset from the call instruction
  - memory locations must be calculated

Avoid using absolute addresses as it is hard to use in other processes. 64-bit systems allow for RIP relative addressing

There exist 'special' byte limitations which prevent certain characters from being used:

- strcpy() stops copying at the null byte
- gets() reads one line at a time, stopping at '\n'
- input may need to be alphanumeric

The following bypasses may be used:

- rewrite shellcode to avoid these characters
  - alternate instructions can achieve a similar result
  - use multiple instructions and ALU operations to construct constants and addresses at run time
- encode shellcode: a 1st stage shellcode decodes a 2nd stage then executes it

## Data Execution Prevention

Originally processors supported only read and write operations, with execution being implied on read

**Translation Lookaside Buffer (TLB)**: a cache for storing translations for the most frequently accessed pages

NX-bit: no-execute bit, prevents code injections

- introduced in 2001 and makes a readable page non-executable
- widely adopted **except for**:
  - low-level IoT processors
  - browsers

JIT-compiled code and code caches imply the following:

- code is continuously generated
- code needs to be executed